In [3]:
import pandas as pd

# Read the Excel file


# Convert to CSV


def lsoa_to_borough_mapping():
    
    data = pd.read_csv('data/secondary_data/lsoa_data/lsoa_data_iadatasheet1.csv')
    if 'Unnamed: 1' in data.columns:
        # Create the 'borough' column by stripping the numeric suffix and any whitespace from the 'Unnamed: 1' column
        data['borough'] = data['Unnamed: 1'].str.extract(r'(^[\D\s]+)').apply(lambda x: x.str.strip())
        
        # Get the position of the 'Unnamed: 1' column
        idx = data.columns.get_loc('Unnamed: 1')
        
        # Rearrange columns to place 'borough' right after 'Unnamed: 1'
        cols = list(data.columns)
        # Move 'borough' to the right position
        new_cols = cols[:idx+1] + ['Borough'] + cols[idx+1:-1]  # assuming 'borough' is the last in the list
        data = data[new_cols]
        
        # Save the modified dataframe back to CSV or return it
        data.to_csv(file_path, index=False)
        return data
    
    
    # Rename 'Unnamed: 1' column to 'LSOA code'
    data.rename(columns={'Unnamed: 1': 'LSOA code'}, inplace=True)
    data = data[['LSOA code', 'Borough']]
    
    data.to_csv('data/lookup_tables/lsoa2borough.csv', index=False)
    # Display the first few rows of the data
    print(data.head())


In [4]:
lsoa_to_borough_mapping

<function __main__.lsoa_to_borough_mapping()>

In [8]:
import os

def list_files(startpath):
    with open('directory_structure.txt', 'w') as f:
        for root, dirs, files in os.walk(startpath):
            level = root.replace(startpath, '').count(os.sep)
            indent = ' ' * 4 * (level)
            f.write('{}{}/\n'.format(indent, os.path.basename(root)))
            subindent = ' ' * 4 * (level + 1)
            for file in files:
                f.write('{}{}\n'.format(subindent, file))

# Replace 'your_path_to_primary_data' with the path to your 'primary_data' directory.
list_files('/Users/banana/Documents/GitHub/DC2-GRP22/data/primary_data')



In [14]:
import os
import pandas as pd

def concatenate_csv_files(base_dir, file_patterns):
    # Create a new directory for concatenated CSVs
    concatenated_dir = os.path.join(base_dir, 'concatenated_csvs')
    os.makedirs(concatenated_dir, exist_ok=True)
    
    # Dictionary to hold the dataframes for each pattern
    dataframes_dict = {pattern: [] for pattern in file_patterns}
    
    # Walk through the directory structure
    for root, _, files in os.walk(base_dir):
        for file in files:
            # Check if the file matches any of the patterns
            for pattern in file_patterns:
                if pattern in file:
                    # Read the CSV file and append to the correct list
                    df = pd.read_csv(os.path.join(root, file))
                    dataframes_dict[pattern].append(df)
                    break
    
    # Now concatenate the dataframes for each pattern and write to a CSV file
    for pattern, dfs in dataframes_dict.items():
        if dfs:  # Check if there are any dataframes to concatenate
            concatenated_df = pd.concat(dfs, ignore_index=True)
            output_file = os.path.join(concatenated_dir, f'{pattern}.csv')
            concatenated_df.to_csv(output_file, index=False)
            print(f'Created file: {output_file} with {len(concatenated_df)} rows.')

# Base directory path
base_dir = '../data/primary_data'

# List of unique patterns to match files for concatenation
file_patterns = [
    'metropolitan-stop-and-search', 'city-of-london-street', 'btp-street', 
    'metropolitan-street', 'city-of-london-stop-and-search', 'metropolitan-outcomes', 
    'city-of-london-outcomes', 'btp-stop-and-search'
]

# Call the function with the base directory and the list of file patterns
concatenate_csv_files(base_dir, file_patterns)



Created file: ../data/primary_data/concatenated_csvs/metropolitan-stop-and-search.csv with 517765 rows.
Created file: ../data/primary_data/concatenated_csvs/city-of-london-street.csv with 25221 rows.
Created file: ../data/primary_data/concatenated_csvs/btp-street.csv with 175708 rows.
Created file: ../data/primary_data/concatenated_csvs/metropolitan-street.csv with 3240987 rows.
Created file: ../data/primary_data/concatenated_csvs/city-of-london-stop-and-search.csv with 7310 rows.
Created file: ../data/primary_data/concatenated_csvs/metropolitan-outcomes.csv with 2556761 rows.
Created file: ../data/primary_data/concatenated_csvs/city-of-london-outcomes.csv with 17825 rows.
Created file: ../data/primary_data/concatenated_csvs/btp-stop-and-search.csv with 38315 rows.


In [13]:
base_dir = '../data/primary_data'
concatenated_dir = os.path.join(base_dir, 'concatenated_csvs')
os.makedirs(concatenated_dir, exist_ok=True)